In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

def value_iteration(
    max_O=5,   # max orders to track (O1,O2,O3)
    max_A=5,   # max cabbages to track (A1,A2)
    p_cancel=0.15,
    gamma=1.0,  
    theta=1e-6,
    max_iter=10000,
    track_deltas=True
):
    """
    NumPy-optimized Value Iteration for the cabbage merchant MDP.
    Uses array indexing instead of dictionary lookups for better performance.
    """
    # Initialize value function and policy using numpy arrays
    # Shape: (max_O+1, max_O+1, max_O+1, max_A+1, max_A+1, 2, 2)
    V = np.zeros((max_O+1, max_O+1, max_O+1, max_A+1, max_A+1, 2, 2))
    policy = np.zeros((max_O+1, max_O+1, max_O+1, max_A+1, max_A+1, 2, 2, 2), dtype=int)
    # policy has extra dimension of size 2 to store (accept, purchase) pairs
    
    # All possible actions
    actions = [(accept, purchase) 
              for accept in [0, 1] 
              for purchase in range(6)]

    def next_state_and_reward(state_idx, action):
        """
        Given state indices and action, return next state indices and rewards.
        """
        O1, O2, O3, A1, A2, c1, c2 = state_idx
        accept, purchase = action

        # Compute immediate reward
        arriving_cabbages = A1 if c1 == 0 else 0
        delivered = min(O1, arriving_cabbages)
        missed = O1 - delivered
        immediate_reward = 4 * delivered - missed - purchase

        # Compute next state components
        next_O1 = O2
        next_O2 = O3
        next_O3 = min(O3 + accept, max_O)
        next_A1 = A2
        next_A2 = min(purchase, max_A)
        next_c1 = c2

        # Return transitions for both possible values of next_c2
        transitions = []
        # Case 1: Not canceled (c2=0)
        s_ok = (next_O1, next_O2, next_O3, next_A1, next_A2, next_c1, 0)
        transitions.append((s_ok, 1-p_cancel, immediate_reward))
        # Case 2: Canceled (c2=1)
        s_cx = (next_O1, next_O2, next_O3, next_A1, next_A2, next_c1, 1)
        transitions.append((s_cx, p_cancel, immediate_reward))

        return transitions

    # Value Iteration main loop
    deltas = []
    for iteration in tqdm(range(max_iter), desc="Value Iteration"):
        delta = 0.0
        # Use numpy's ndindex to iterate over all state indices
        for state_idx in np.ndindex(V.shape):
            old_value = V[state_idx]
            best_value = float('-inf')
            best_act = (0, 0)

            for action in actions:
                outcomes = next_state_and_reward(state_idx, action)
                q_sa = 0.0
                for (next_state_idx, prob, reward) in outcomes:
                    q_sa += prob * (reward + gamma * V[next_state_idx])

                if q_sa > best_value:
                    best_value = q_sa
                    best_act = action

            delta = max(delta, abs(best_value - old_value))
            V[state_idx] = best_value
            policy[state_idx] = best_act

        deltas.append(delta)
        if delta < theta:
            print(f"Converged in {iteration} iterations.")
            break

    if track_deltas:
        return V, policy, deltas
    else:
        return V, policy, None

if __name__ == "__main__":
    # Run the optimized value iteration
    V, policy, deltas = value_iteration(
        max_O=5,
        max_A=5,
        p_cancel=0.15,
        gamma=1.0,
        theta=1e-8,
        max_iter=10000,
        track_deltas=True
    )

    # Plot convergence
    plt.figure(figsize=(8, 5))
    plt.plot(deltas, label="Delta per iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Max Value Update (Delta)")
    plt.title("Value Iteration Convergence (NumPy Optimized)")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Show optimal actions for sample states
    sample_states = [
        (0,0,0,0,0,0,0),
        (1,0,0,1,0,0,0),
        (2,2,0,2,1,0,1)
    ]
    for s in sample_states:
        print(f"State {s}: best action = {tuple(policy[s])}, value = {V[s]:.2f}")

In [ ]:

# After running value_iteration:
np.save("value_function.npy", V)
np.save("policy.npy", policy)


In [3]:
import numpy as np

# Later, you can load them without re-running everything:
V = np.load("value_function.npy")
policy = np.load("policy.npy")

In [5]:
import numpy as np
import random

# 1) Load the stored policy
policy = np.load("policy.npy")

# 2) Define a simulation function matching your MDP logic.
def simulate_policy_1day_cancellation(policy, 
                                      n_days=1000, 
                                      p_cancel=0.15, 
                                      max_O=5, 
                                      max_A=5, 
                                      seed=42):
    """
    Simulates day-by-day using a policy array:
      policy.shape = (max_O+1, max_O+1, max_O+1, max_A+1, max_A+1, 2, 2, 2)

    State indices: (O1, O2, O3, A1, A2, c1, c2).
      c1=1 => tomorrow's arrival is canceled, c1=0 => it will arrive
      c2=1 => day-after-tomorrow's arrival is canceled, c2=0 => it will arrive

    The policy entry: policy[O1, O2, O3, A1, A2, c1, c2] = [accept, purchase]
    where 'accept' in {0,1}, 'purchase' in range(6) if we followed your code.

    Returns:
      total_profit, daily_profits
    """
    random.seed(seed)
    
    # Start state: (0,0,0,0,0, c1=0, c2=0)
    state = (0, 0, 0, 0, 0, 0, 0)
    (O1, O2, O3, A1, A2, c1, c2) = state

    daily_profits = []
    total_profit = 0.0

    for day in range(n_days):
        # 1) Retrieve policy decision
        #    If indices go out of bounds, clamp them so we don't crash.
        #    (Or do a check; here we clamp to ensure we don't exceed array dims.)
        sO1 = min(O1, max_O)
        sO2 = min(O2, max_O)
        sO3 = min(O3, max_O)
        sA1 = min(A1, max_A)
        sA2 = min(A2, max_A)
        sc1 = 1 if c1 != 0 else 0
        sc2 = 1 if c2 != 0 else 0

        # policy array holds [accept, purchase]
        accept, purchase = policy[sO1, sO2, sO3, sA1, sA2, sc1, sc2]

        # 2) If c1=0 => A1 arrives, else 0
        arriving_cabbages = A1 if (c1 == 0) else 0

        # Deliver to O1
        delivered = min(O1, arriving_cabbages)
        missed = O1 - delivered

        # Daily profit
        day_profit = 4*delivered - missed - purchase
        total_profit += day_profit
        daily_profits.append(day_profit)

        # Shift states:
        next_O1 = O2
        next_O2 = O3
        next_O3 = O3 + accept
        if next_O3 > max_O:
            next_O3 = max_O

        next_A1 = A2
        next_A2 = purchase if purchase <= max_A else max_A

        next_c1 = c2  # shift c2 -> c1

        # Draw new c2
        if random.random() < p_cancel:
            next_c2 = 1
        else:
            next_c2 = 0

        # Update state
        state = (next_O1, next_O2, next_O3, next_A1, next_A2, next_c1, next_c2)
        (O1, O2, O3, A1, A2, c1, c2) = state

    return total_profit, daily_profits


# 3) Test with multiple day ranges
day_ranges = [100, 1000, 5000, 10000, 20000, 50000, 100000]
for n in day_ranges:
    total, daily_list = simulate_policy_1day_cancellation(
        policy=policy,
        n_days=n,
        p_cancel=0.15,
        max_O=5, 
        max_A=5, 
        seed=42
    )
    avg_profit = total / n
    print(f"For n_days={n}, total profit={total:.2f}, average profit/day={avg_profit:.3f}")


For n_days=100, total profit=551.00, average profit/day=5.510
For n_days=1000, total profit=5774.00, average profit/day=5.774
For n_days=5000, total profit=28293.00, average profit/day=5.659
For n_days=10000, total profit=55823.00, average profit/day=5.582
For n_days=20000, total profit=111800.00, average profit/day=5.590
For n_days=50000, total profit=280043.00, average profit/day=5.601
For n_days=100000, total profit=557676.00, average profit/day=5.577
